# #The first Function RoomCountGumTreeScraper performs the scrap

In [58]:
def RoomCountGumTreeScraper(x,counter):

    #Import all the prerequiste libraries
    
    import urllib2
    from bs4 import BeautifulSoup
    from selenium import webdriver
    import pandas as pd
    import numpy as np
    import re
    #The below librarirs are used to write the dataframe to a google sheet which 
    #will then be linked to through the open data site
    import datetime

    #Use selenium to execute the javascript and scrape the data
    driver = webdriver.Chrome()
    driver.get(x)
    
    #Use BeautifulSoup to parse the webpage
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    #Search the elements where links are present
    #In Gumetre to the left side of the page the total nunber of rooms for the searched suburub is always diplayed
    #Have scraped the data from there
    
    g_data = soup.findAll('a',{'class':'srp-list-filter__item-link link link--no-underline'})

    #initialise total_rooms
    Total_rooms=[]
    for match in g_data:
        Total_rooms.append(match.text)


    
    df = pd.DataFrame(np.array(Total_rooms))
    df.columns=["RawData"]
    #If you want to view the Rawdata just print the above df
    
    
    #The actual number of rooms within the parenthesis is extracted
    df['No_of_rooms']=df['RawData'].str.replace(r'[^(]*\(|\)[^)]*', '')
    
    #Extract the type of room basically exclude everything after a parenthesis is detected
    df['Type_of_Department']=df['RawData'].str.replace(r"\(.*\)","")



    Final_df=df[df['RawData'].str.contains('Bedroom',na = False)]

    del Final_df['RawData']
    Final_df["Capture_Time"]=datetime.datetime.now()
    Final_df.set_index('No_of_rooms')

    DFtoGoogleSheet(Final_df,counter)



# #The function DFtoGoogleSheet used this to transfer to the Google Doc

In [62]:
def DFtoGoogleSheet(DftobeTransferred,counter):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials
    from gspread_dataframe import get_as_dataframe, set_with_dataframe

    # use creds to create a client to interact with the Google Drive API
    scope = ['https://spreadsheets.google.com/feeds']
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds) 
    
    #Open Sesame!
    if (counter == 1): 
        sheet = client.open("RWA_InnerWest").get_worksheet(0)
    elif (counter == 2):
        sheet = client.open("RWA_InnerWest").get_worksheet(1)
    else:
        sheet = client.open("RWA_InnerWest").get_worksheet(2)

    #Write DataFrame to Sheet    
    set_with_dataframe(sheet, DftobeTransferred)



# The main function that calls the above to perform the scraping,the list of suburb urls are present in the list below:

In [68]:
#The main execution of the py script

links_of_suburbs=("https://www.gumtree.com.au/s-flatshare-houseshare/granville-sydney/c18294l3003933?sort=rank","https://www.gumtree.com.au/s-flatshare-houseshare/sydney/parramatta/k0c18294l3003435?sort=rank","https://www.gumtree.com.au/s-flatshare-houseshare/sydney/newtown/k0c18294l3003435?sort=rank")

counter=1
for x in links_of_suburbs:
    print "The suburb url in consderation is",x
    print "the counter is ",counter
    RoomCountGumTreeScraper(x,counter)
    counter=counter+1
    


    

The suburb url in consderation is https://www.gumtree.com.au/s-flatshare-houseshare/granville-sydney/c18294l3003933?sort=rank
the counter is  1
<type 'list'>


C:\Users\sidha\Anaconda2\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


The suburb url in consderation is https://www.gumtree.com.au/s-flatshare-houseshare/sydney/parramatta/k0c18294l3003435?sort=rank
the counter is  2
<type 'list'>
The suburb url in consderation is https://www.gumtree.com.au/s-flatshare-houseshare/sydney/newtown/k0c18294l3003435?sort=rank
the counter is  3
<type 'list'>
